# * Ex-D Data

## Parameter

In [6]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [7]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-04-25, 10:01:06

   -> Summany DataFrame : 128 rows, 20 columns

TDMDBPR : Disconnected


In [8]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250422,DB1R000100,Prepay Revenue,2025-04-22,20250413,20250422,"526,831,766","1,053,663,533","1,052,791,099","1,052,791,099",0,"11,289,175",4,170,1850
1,20250422,DB2R000100,Postpaid Revenue,2025-04-22,20250413,20250422,"812,316,636","1,624,633,272","1,623,252,862","1,623,252,862",0,"28,819,157",4,170,1836
2,20250422,DB2R010100,Postpaid Revenue B2C,2025-04-22,20250413,20250422,"791,110,518","1,582,221,036","1,580,846,327","1,580,846,327",0,"28,726,441",4,170,1836
3,20250422,DB2R020100,Postpaid Revenue B2B,2025-04-22,20250413,20250422,"21,206,118","42,412,236","42,406,535","42,406,535",0,"92,716",4,170,1830


In [9]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250422,DB1S000500,Prepay Usage Subs,2025-04-22,20250413,20250422,"113,319,319","226,638,638","203,863,456","203,863,456",0,"2,281,320",4,170,1850
1,20250422,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-04-22,20250413,20250422,"100,157,019","105,907,622","109,995,390","110,277,966",0,"2,460,568",4,170,1740
2,20250422,DB2S000500,Postpaid Active Subs,2025-04-22,20250413,20250422,"58,031,750","116,063,500","104,364,850","104,364,850",0,"1,750,458",4,170,1850
3,20250422,DB2S010500,Postpaid Active Subs B2C,2025-04-22,20250413,20250422,"51,121,947","102,243,894","91,945,691","91,945,691",0,"1,575,665",4,170,1850
4,20250422,DB2S020400,Postpaid Active Sub B2B,2025-04-22,20250413,20250422,"6,909,803","13,819,606","12,419,159","12,419,159",0,"174,793",4,170,1808


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [10]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-04-25, 10:01:09

   -> Daily DataFrame : 90 rows, 16 columns

TDMDBPR : Disconnected


In [11]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250422,DB1R000100,Prepay Revenue,20250413,2025-04-22,"52,617,783","105,235,566","105,148,619","105,148,619",0,"1,130,314",0,0,4,170,185
1,20250422,DB1R000100,Prepay Revenue,20250414,2025-04-22,"49,106,476","98,212,951","98,138,254","98,138,254",0,"1,106,194",0,0,4,170,185
2,20250422,DB1R000100,Prepay Revenue,20250415,2025-04-22,"49,605,471","99,210,942","99,121,423","99,121,423",0,"1,091,940",0,0,4,170,185
3,20250422,DB1R000100,Prepay Revenue,20250416,2025-04-22,"53,196,521","106,393,042","106,304,710","106,304,710",0,"1,130,089",0,0,4,170,185
4,20250422,DB1R000100,Prepay Revenue,20250417,2025-04-22,"54,043,511","108,087,022","107,999,932","107,999,932",0,"1,151,668",0,0,4,170,185
5,20250422,DB1R000100,Prepay Revenue,20250418,2025-04-22,"53,658,151","107,316,302","107,231,118","107,231,118",0,"1,118,994",0,0,4,170,185
6,20250422,DB1R000100,Prepay Revenue,20250419,2025-04-22,"52,332,240","104,664,479","104,574,941","104,574,941",0,"1,111,991",0,0,4,170,185
7,20250422,DB1R000100,Prepay Revenue,20250420,2025-04-22,"54,071,040","108,142,081","108,054,022","108,054,022",0,"1,146,538",0,0,4,170,185
8,20250422,DB1R000100,Prepay Revenue,20250421,2025-04-22,"55,363,443","110,726,885","110,635,130","110,635,130",0,"1,161,318",0,0,4,170,185
9,20250422,DB1R000100,Prepay Revenue,20250422,2025-04-22,"52,837,131","105,674,262","105,582,950","105,582,950",0,"1,140,129",0,0,4,170,185


In [12]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250422,DB2R000100,Postpaid Revenue,20250413,2025-04-22,"84,672,588","169,345,176","169,192,961","169,192,961",0,"3,174,570",0,0,4,170,183
1,20250422,DB2R000100,Postpaid Revenue,20250414,2025-04-22,"78,494,695","156,989,390","156,835,022","156,835,022",0,"2,844,605",0,0,4,170,183
2,20250422,DB2R000100,Postpaid Revenue,20250415,2025-04-22,"80,393,629","160,787,258","160,639,597","160,639,597",0,"2,870,098",0,0,4,170,183
3,20250422,DB2R000100,Postpaid Revenue,20250416,2025-04-22,"80,395,169","160,790,337","160,653,275","160,653,275",0,"2,945,108",0,0,4,170,183
4,20250422,DB2R000100,Postpaid Revenue,20250417,2025-04-22,"81,062,686","162,125,371","161,987,209","161,987,209",0,"2,763,153",0,0,4,170,183
5,20250422,DB2R000100,Postpaid Revenue,20250418,2025-04-22,"82,806,473","165,612,946","165,484,133","165,484,133",0,"2,789,006",0,0,4,170,183
6,20250422,DB2R000100,Postpaid Revenue,20250419,2025-04-22,"81,842,320","163,684,640","163,549,202","163,549,202",0,"2,872,758",0,0,4,170,185
7,20250422,DB2R000100,Postpaid Revenue,20250420,2025-04-22,"79,430,927","158,861,854","158,739,762","158,739,762",0,"2,862,001",0,0,4,170,185
8,20250422,DB2R000100,Postpaid Revenue,20250421,2025-04-22,"81,661,596","163,323,192","163,187,842","163,187,842",0,"2,854,194",0,0,4,170,183
9,20250422,DB2R000100,Postpaid Revenue,20250422,2025-04-22,"81,556,554","163,113,108","162,983,860","162,983,860",0,"2,843,665",0,0,4,170,185


In [13]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250422,DB2R010100,Postpaid Revenue B2C,20250413,2025-04-22,"82,958,015","165,916,030","165,763,918","165,763,918",0,"3,159,312",0,0,4,170,183
1,20250422,DB2R010100,Postpaid Revenue B2C,20250414,2025-04-22,"77,170,485","154,340,971","154,188,655","154,188,655",0,"2,832,045",0,0,4,170,183
2,20250422,DB2R010100,Postpaid Revenue B2C,20250415,2025-04-22,"77,192,352","154,384,705","154,237,155","154,237,155",0,"2,859,774",0,0,4,170,183
3,20250422,DB2R010100,Postpaid Revenue B2C,20250416,2025-04-22,"78,332,139","156,664,278","156,527,703","156,527,703",0,"2,935,394",0,0,4,170,183
4,20250422,DB2R010100,Postpaid Revenue B2C,20250417,2025-04-22,"77,712,247","155,424,494","155,286,689","155,286,689",0,"2,755,886",0,0,4,170,183
5,20250422,DB2R010100,Postpaid Revenue B2C,20250418,2025-04-22,"79,234,979","158,469,958","158,341,446","158,341,446",0,"2,782,626",0,0,4,170,183
6,20250422,DB2R010100,Postpaid Revenue B2C,20250419,2025-04-22,"80,926,589","161,853,179","161,718,209","161,718,209",0,"2,867,101",0,0,4,170,185
7,20250422,DB2R010100,Postpaid Revenue B2C,20250420,2025-04-22,"78,606,529","157,213,058","157,091,545","157,091,545",0,"2,858,251",0,0,4,170,185
8,20250422,DB2R010100,Postpaid Revenue B2C,20250421,2025-04-22,"79,589,469","159,178,938","159,044,563","159,044,563",0,"2,843,357",0,0,4,170,183
9,20250422,DB2R010100,Postpaid Revenue B2C,20250422,2025-04-22,"79,387,713","158,775,426","158,646,444","158,646,444",0,"2,832,696",0,0,4,170,185


In [14]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250422,DB2R020100,Postpaid Revenue B2B,20250413,2025-04-22,"1,714,573","3,429,146","3,429,043","3,429,043",0,"15,258",0,0,4,170,183
1,20250422,DB2R020100,Postpaid Revenue B2B,20250414,2025-04-22,"1,324,210","2,648,419","2,646,367","2,646,367",0,"12,559",0,0,4,170,183
2,20250422,DB2R020100,Postpaid Revenue B2B,20250415,2025-04-22,"3,201,277","6,402,553","6,402,442","6,402,442",0,"10,324",0,0,4,170,183
3,20250422,DB2R020100,Postpaid Revenue B2B,20250416,2025-04-22,"2,063,029","4,126,059","4,125,571","4,125,571",0,"9,714",0,0,4,170,183
4,20250422,DB2R020100,Postpaid Revenue B2B,20250417,2025-04-22,"3,350,439","6,700,877","6,700,520","6,700,520",0,"7,267",0,0,4,170,183
5,20250422,DB2R020100,Postpaid Revenue B2B,20250418,2025-04-22,"3,571,494","7,142,988","7,142,686","7,142,686",0,"6,380",0,0,4,170,183
6,20250422,DB2R020100,Postpaid Revenue B2B,20250419,2025-04-22,"915,730","1,831,461","1,830,993","1,830,993",0,"5,657",0,0,4,170,183
7,20250422,DB2R020100,Postpaid Revenue B2B,20250420,2025-04-22,"824,398","1,648,796","1,648,217","1,648,217",0,"3,750",0,0,4,170,183
8,20250422,DB2R020100,Postpaid Revenue B2B,20250421,2025-04-22,"2,072,127","4,144,254","4,143,279","4,143,279",0,"10,837",0,0,4,170,183
9,20250422,DB2R020100,Postpaid Revenue B2B,20250422,2025-04-22,"2,168,841","4,337,682","4,337,417","4,337,417",0,"10,969",0,0,4,170,183


In [15]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]